In [6]:
#STEP one fit good model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ordinal_xai.utils import load_data
from sklearn.model_selection import train_test_split

X,y = load_data("ordinal_xai/data/winequality-red.csv")#,sep=",")#,label_map={"unacc":0,"acc":1,"good":2,"vgood":3})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from ordinal_xai.models import CLM,ONN,OBD,OGBoost
from ordinal_xai.utils import evaluate_ordinal_model


model = OBD(base_classifier="xgb",decomposition_type="one-vs-following")

model.fit(X_train,y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)
y_pred_train = model.predict(X_train)
y_pred_proba_train = model.predict_proba(X_train)
print(evaluate_ordinal_model(y_train,y_pred_train,y_pred_proba_train))
print(evaluate_ordinal_model(y_test,y_pred,y_pred_proba))
#refit model on full data
model.fit(X,y)
y_pred=model.predict(X)
y_pred_proba=model.predict_proba(X)



c:\Users\wankm\Projects\ordinal_xai\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [11:00:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\wankm\Projects\ordinal_xai\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [11:00:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\wankm\Projects\ordinal_xai\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [11:00:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarn

{'accuracy': 0.800625488663018, 'adjacent_accuracy': 0.9843627834245504, 'mze': 0.19937451133698203, 'mae': 0.21579358874120408, 'mse': 0.2501954652071931, 'weighted_kappa_quadratic': 0.7844599227535709, 'weighted_kappa_linear': 0.7261909009535895, 'cem': 0.8342706258436001, 'spearman_correlation': 0.7712083357544592, 'kendall_tau': 0.7455880703163329, 'ranked_probability_score': 0.1766763479920882, 'ordinal_weighted_ce_linear': 0.549229864221745, 'ordinal_weighted_ce_quadratic': 0.7140017688787844}
{'accuracy': 0.63125, 'adjacent_accuracy': 0.9875, 'mze': 0.36875, 'mae': 0.38125, 'mse': 0.40625, 'weighted_kappa_quadratic': 0.6287239169626759, 'weighted_kappa_linear': 0.5042791477258297, 'cem': 0.7180186382218141, 'spearman_correlation': 0.6450715185465562, 'kendall_tau': 0.6010054406985176, 'ranked_probability_score': 0.2745957430777267, 'ordinal_weighted_ce_linear': 0.7472586697604687, 'ordinal_weighted_ce_quadratic': 0.9211852209318444}


c:\Users\wankm\Projects\ordinal_xai\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [11:00:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\wankm\Projects\ordinal_xai\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [11:00:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\wankm\Projects\ordinal_xai\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [11:00:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarn

In [32]:
#generate set of observations that is uniform distributed across ranks, generator yield
import math
def generate_observations(y,n=100,balance_classes=True):
    #set random seed
    np.random.seed(42)
    if balance_classes == False:
        for i in range(n):
            yield np.random.randint(0,len(y))
        return
    yc = y.copy()
    classes = np.unique(yc)
    for i in range(n):
        random_class = np.random.choice(classes)
        try:
            obs = np.random.choice(np.where(yc==random_class)[0])
        except:
            "not enough samples for class",random_class
            return
        #yc[obs] = -1
        yield obs

def nonzero_coefs(results):
    if 'lower_coef' in results:
        share_lower_nonzeroc =np.count_nonzero(results['lower_coef'])/len(results['lower_coef'])
        if 'higher_coef' not in results:
            return share_lower_nonzeroc
    if 'higher_coef' in results:
        share_higher_nonzeroc =np.count_nonzero(results['higher_coef'])/len(results['higher_coef'])
        if 'lower_coef' not in results:
            return share_higher_nonzeroc
    return np.mean([share_lower_nonzeroc, share_higher_nonzeroc])

In [ ]:
from ordinal_xai.interpretation.lime import LIME
from  time import time
count = 0
lime_tree_3 = LIME(model, X, y, sampling="uniform", model_type="decision_tree",max_samples=500)
lime_tree_2 = LIME(model, X, y, sampling="uniform", model_type="decision_tree",max_depth=2,max_samples=500)
C_values = np.logspace(-3,3,7)
lime_L1 = {c:LIME(model, X, y, sampling="uniform", model_type="logistic",penalty="l1",C=c,solver='liblinear',max_samples=500) for c in C_values}
lime_L2 = {c:LIME(model, X, y, sampling="uniform", model_type="logistic",penalty="l2",C=c,max_samples=500) for c in C_values}
lime_elastic = {c:LIME(model, X, y, sampling="uniform", model_type="logistic",penalty="elasticnet",C=c,solver='saga',l1_ratio=0.5,max_samples=500) for c in C_values}
results = pd.DataFrame(columns=["idx","true_rank","predicted_rank","fidelity_out_bce_tree3","fidelity_out_01_tree3","fidelity_out_bce_tree2","fidelity_out_01_tree2","fidelity_out_bce_L1","fidelity_out_01_L1","fidelity_out_bce_L2","fidelity_out_01_L2","fidelity_out_bce_elastic","fidelity_out_01_elastic","nonzero_coefs_L1","nonzero_coefs_L2","nonzero_coefs_elastic"])

for idx in generate_observations(y_pred,100,balance_classes=False):
    #remove timing
    count += 1

    results_tree_3 = lime_tree_3.explain(observation_idx=idx)
    fidelity_out_bce_tree3 = np.mean([results_tree_3["higher_fidelity_out_bce"],results_tree_3["lower_fidelity_out_bce"]])
    fidelity_out_01_tree3 = np.mean([results_tree_3["higher_fidelity_out_01"],results_tree_3["lower_fidelity_out_01"]])
    results_tree_2 = lime_tree_2.explain(observation_idx=idx)
    fidelity_out_bce_tree2 = np.mean([results_tree_2["higher_fidelity_out_bce"],results_tree_2["lower_fidelity_out_bce"]])
    fidelity_out_01_tree2 = np.mean([results_tree_2["higher_fidelity_out_01"],results_tree_2["lower_fidelity_out_01"]])
    results_L1 = {c:lime_L1[c].explain(observation_idx=idx,plot=False) for c in C_values}
    fidelity_out_bce_L1 = {c:np.mean([results_L1[c]["higher_fidelity_out_bce"],results_L1[c]["lower_fidelity_out_bce"]]) for c in C_values}
    fidelity_out_01_L1 = {c:np.mean([results_L1[c]["higher_fidelity_out_01"],results_L1[c]["lower_fidelity_out_01"]]) for c in C_values}
    nonzero_coefs_L1 = {c:nonzero_coefs(results_L1[c]) for c in C_values}
    results_L2 = {c:lime_L2[c].explain(observation_idx=idx,plot=False) for c in C_values}
    fidelity_out_bce_L2 = {c:np.mean([results_L2[c]["higher_fidelity_out_bce"],results_L2[c]["lower_fidelity_out_bce"]]) for c in C_values}
    fidelity_out_01_L2 = {c:np.mean([results_L2[c]["higher_fidelity_out_01"],results_L2[c]["lower_fidelity_out_01"]]) for c in C_values}
    nonzero_coefs_L2 = {c:nonzero_coefs(results_L2[c]) for c in C_values}
    results_elastic = {c:lime_elastic[c].explain(observation_idx=idx,plot=False) for c in C_values}
    fidelity_out_bce_elastic = {c:np.mean([results_elastic[c]["higher_fidelity_out_bce"],results_elastic[c]["lower_fidelity_out_bce"]]) for c in C_values}
    fidelity_out_01_elastic = {c:np.mean([results_elastic[c]["higher_fidelity_out_01"],results_elastic[c]["lower_fidelity_out_01"]]) for c in C_values}
    nonzero_coefs_elastic = {c:nonzero_coefs(results_elastic[c]) for c in C_values}
    
print(count)

decision tree runtime:  0.15327215194702148
L1 runtime:  4.105225563049316
L2 runtime:  1.6071240901947021
elastic runtime:  7.307039260864258
1


In [ ]:
def nonzero_coefs(results):
    if 'lower_coef' in results:
        share_lower_nonzeroc =np.count_nonzero(results['lower_coef'])/len(results['lower_coef'])
        if 'higher_coef' not in results:
            return share_lower_nonzeroc
    if 'higher_coef' in results:
        share_higher_nonzeroc =np.count_nonzero(results['higher_coef'])/len(results['higher_coef'])
        if 'lower_coef' not in results:
            return share_higher_nonzeroc
    return np.mean([share_lower_nonzeroc, share_higher_nonzeroc])
    

0.5909090909090908